In [116]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
import stim

In [117]:
def stim_to_qiskit_circuit(stim_circuit,skipMeas=False):
    """
    Converts a Stim Circuit to a Qiskit QuantumCircuit.

    Parameters:
        stim_circuit: stim.Circuit
            A Stim circuit object.

    Returns:
        QuantumCircuit:
            A Qiskit QuantumCircuit equivalent to the Stim circuit.
    """
    num_qubits = stim_circuit.num_qubits
    qiskit_circuit = QuantumCircuit(num_qubits)

    # Parse each instruction in the Stim circuit
    for instruction in stim_circuit:
        gate = instruction.name
        L=instruction.target_groups()[0]
        qubits = [ x.qubit_value for x in L]
        #print(gate,'targets',qubits)

        if gate == "H":  # Hadamard
            qiskit_circuit.h(qubits[0])
        elif gate == "S":  # Phase (S)
            qiskit_circuit.s(qubits[0])
        elif gate in ["CNOT", "CX"]:  # Controlled-NOT
            qiskit_circuit.cx(qubits[0],qubits[1])
        elif gate == "X":  # Pauli-X
            qiskit_circuit.x(qubits[0])
        elif gate == "Z":  # Pauli-Z
            qiskit_circuit.z(qubits[0])
        elif gate == "M":  # Measurement
            if skipMeas: continue
            assert len(instruction.target_groups()) == num_qubits
            qiskit_circuit.measure_all() # not universal solution
        else:
            raise ValueError(f"Unsupported gate: {gate}")

    return qiskit_circuit


In [118]:
# Create a Stim circuit
stim_circuit = stim.Circuit("""
H 0
S 1
CNOT 0 1
X 2
M 0 1 2
""")
# 
# Convert Stim circuit to Qiskit circuit
qc = stim_to_qiskit_circuit(stim_circuit,skipMeas=True)
print(qc.draw())


     ┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ S ├┤ X ├
     ├───┤└───┘
q_2: ┤ X ├─────
     └───┘     


In [119]:
def rnd_clifford_state(nq):
    """
    Generates a random Clifford state for a given number of qubits, transpiles it, and computes its statevector.

    Parameters:
        nq (int): Number of qubits for the Clifford state.

    Returns:
        tuple:
            - qcT (QuantumCircuit): Transpiled Qiskit circuit representing the random Clifford state.
            - state (Statevector): Statevector of the generated Clifford state.
    """
    # Generate a random stabilizer tableau using Stim
    tableau = stim.Tableau.random(nq)
    
    # Convert the tableau to a Stim circuit using Gaussian elimination (efficient stabilizer simulation)
    qcSt = tableau.to_circuit(method="elimination")
    
    # Debugging option to view the Stim circuit diagram
    # print(qcSt.diagram())
    
    # Convert the Stim circuit to a Qiskit QuantumCircuit (skip measurement operations)
    qc = stim_to_qiskit_circuit(qcSt, skipMeas=True)
    
    # Initialize a Qiskit simulator backend
    backend = AerSimulator()
    
    # Transpile the Qiskit circuit to reduce 1-q gates
    qcT = transpile(qc, backend=backend, optimization_level=1)
    
    # Compute the statevector representation of the transpiled circuit
    state = Statevector.from_instruction(qcT)
    return qcT, state

qc,state=rnd_clifford_state(3)
print(qc.draw())

     ┌───┐┌───┐          ┌───┐                                   
q_0: ┤ X ├┤ H ├──■───────┤ X ├───────────────────────────────────
     └─┬─┘└───┘┌─┴─┐┌───┐└─┬─┘┌──────────────┐┌───┐    ┌───┐     
q_1: ──■───────┤ X ├┤ H ├──■──┤ U2(π/2,-π/2) ├┤ X ├────┤ S ├─────
               └───┘└───┘     └──────────────┘└─┬─┘┌───┴───┴────┐
q_2: ───────────────────────────────────────────■──┤ U2(0,-π/2) ├
                                                   └────────────┘


In [120]:
for _ in range(10): rnd_clifford_state(6)